In [1]:
import numpy as np
from scipy import stats
import polars as pl

In [ ]:
n_subj = 50
trial_types = ["target", "lure"]
item_types = ["word", "picture"]
n_type = 2
n_item = 2
n_time = 3
study_times = [1, 2, 4]
time_lure_strength = [0, -0.2, -0.3]
time_target_strength = [1, 1.2, 1.3]
item_lure_dev = [0.1, -0.1]
item_target_dev = [-0.1, 0.1]
n_trial = 100

In [ ]:
rng = np.random.default_rng(1)
subj_time_dev = rng.normal(loc=0, scale=0.1, size=(n_subj, n_time))
subj_item_dev = rng.normal(loc=0, scale=0.1, size=(n_subj, n_item))
subj_target_dev = rng.normal(loc=0, scale=0.1, size=n_subj)
subj_lure_dev = rng.normal(loc=0, scale=0.1, size=n_subj)
subj_time_target_dev = rng.normal(loc=0, scale=0.05, size=(n_subj, n_time))
subj_time_lure_dev = rng.normal(loc=0, scale=0.05, size=(n_subj, n_time))
subj_item_target_dev = rng.normal(loc=0, scale=0.05, size=(n_subj, n_item))
subj_item_lure_dev = rng.normal(loc=0, scale=0.05, size=(n_subj, n_item))
d = {
    "subject": [],
    "trial": [],
    "study_time": [],
    "item_type": [],
    "trial_type": [],
    "response": [],
}
sd = 0.8
criterion = [0.5, 0.6, 0.7]
for i in range(n_subj):
    trial = 0
    for j in range(n_time):
        for k in range(n_item):
            mu_lure = (
                time_lure_strength[j]
                + item_lure_dev[k]
                + subj_time_dev[i, j]
                + subj_item_dev[i, k]
                + subj_lure_dev[i]
                + subj_time_lure_dev[i, j]
                + subj_item_lure_dev[i, k]
            )
            mu_target = (
                time_target_strength[j]
                + item_target_dev[k]
                + subj_time_dev[i, j]
                + subj_item_dev[i, k]
                + subj_target_dev[i]
                + subj_time_target_dev[i, j]
                + subj_item_target_dev[i, k]
            )
            lure_strength = rng.normal(loc=mu_lure, scale=sd, size=n_trial)
            target_strength = rng.normal(loc=mu_target, scale=sd, size=n_trial)
            for l in range(n_type):
                trial_type = trial_types[l]
                if trial_type == "target":
                    response = np.where(target_strength > criterion[j], 1, 0)
                else:
                    response = np.where(lure_strength > criterion[j], 1, 0)
                for m in range(n_trial):
                    trial += 1
                    d["subject"].append(f"subj{i + 1:02}")
                    d["trial"].append(trial)
                    d["study_time"].append(study_times[j])
                    d["item_type"].append(item_types[k])
                    d["trial_type"].append(trial_type)
                    d["response"].append(response[m])
df = pl.DataFrame(d)
df.write_csv("gen_recog.csv")
df

subject,trial,study_time,item_type,trial_type,response
str,i64,i64,str,str,i64
"""subj01""",1,1,"""word""","""target""",0
"""subj01""",2,1,"""word""","""target""",0
"""subj01""",3,1,"""word""","""target""",1
"""subj01""",4,1,"""word""","""target""",1
"""subj01""",5,1,"""word""","""target""",1
…,…,…,…,…,…
"""subj50""",1196,4,"""picture""","""lure""",0
"""subj50""",1197,4,"""picture""","""lure""",0
"""subj50""",1198,4,"""picture""","""lure""",0
